In [54]:
from ase.build import graphene  # Importa el módulo para construir estructuras de grafeno en ASE.
from ase.io import write       # Importa la función para exportar estructuras a diferentes formatos (como Quantum ESPRESSO).
from ase import Atoms          # Importa la clase Atoms para crear y manipular átomos individuales o moléculas.

from ase.visualize import view # Importa la herramienta para visualizar la estructura

import numpy as np             # Importa numpy para trabajar con operaciones matemáticas y vectores.


# Hoja de grafeno con Hidrogeno en sobre el primer atomo de carbono

In [55]:
unit_cell = graphene() # La función graphene() genera una celda unitaria estándar de grafeno con una constante de red predeterminada (a = 2.46 Å).
#view(unit_cell)

supercell_over = unit_cell.repeat((3,3,1)) # La función repeat((3, 3, 1)) repite la celda unitaria en las direcciones x e y para formar una supercelda de 3x3 hexágonos.
#view(supercell_over)

# Selecciona el primer átomo de carbono de la supercelda (índice 0) y coloca un átomo de hidrógeno justo encima de él.
# El desplazamiento en el eje z es de 1.1 Å para evitar superposición.

hydrogen_1_position = supercell_over[0].position + np.array([0, 0, 1.1])

# Añadir los átomos de hidrógeno
# Usa la clase Atoms para definir un átomo de hidrógeno en la posición calculada y lo agrega a la supercelda.

supercell_over += Atoms("H", positions=[hydrogen_1_position])

#view(supercell_over) # Vista antes de central la molécula en el centro del espacio generado


cell_params = [list(supercell_over.cell[0]),
				list(supercell_over.cell[1]),
				[0.0,0.0,15.0]]

supercell_over.cell[2][2] = 15.0

supercell_over.positions[:,2] += (cell_params[2][2]/2) - supercell_over.get_center_of_mass()[2]

view(supercell_over) # Molécula centrada en el espacio generado

<Popen: returncode: None args: ['/home/hernan-barquero/anaconda3/bin/python'...>

In [56]:
# Genera un archivo de entrada para Quantum ESPRESSO con la configuración donde el hidrógeno está sobre el átomo de carbono.
write("graphene_h_over_c.in", supercell_over, format="espresso-in", 
	pseudopotentials={
    "C": "C.pbe-n-kjpaw_psl.1.0.0.UPF",  # Especifica el pseudopotencial para carbono.
    "H": "H.pbe-kjpaw_psl.1.0.0.UPF"     # Especifica el pseudopotencial para hidrógeno.
}, input_data={
    "control": {
        "calculation": "relax",           # Indica que se hará una relajación de la estructura.
        "prefix": "graphene_h_over_c",    # Prefijo para los archivos de salida generados por Quantum ESPRESSO.
        "outdir": ".",             # Carpeta donde se guardarán los archivos temporales de Quantum ESPRESSO.
        "pseudo_dir": "/home/ebarquero/practicas/p2/pseudo/",          # Carpeta donde se encuentran los pseudopotenciales.
		"tstress": True,
		"tprnfor": True
    },
    "system": {
        "ecutwfc": 60,                    # Corte de energía para las funciones de onda (en Ry).
        "ecutrho": 600,                   # Corte de energía para la densidad de carga (en Ry).
        "occupations": "smearing",        # Usa un método de smearing para las ocupaciones electrónicas.
        "smearing": "mp",                 # Método de smearing: Methfessel-Paxton.
        "degauss": 0.02 ,                  # Ancho del smearing en Ry.
		"ibrav":0, 							# Define una celda generica
    },
    "electrons": {
        "conv_thr": 1e-6                  # Criterio de convergencia para las iteraciones electrónicas.
    }
}, kpts = (3,3,3),koffset = (0,0,0))

# Hoja de grafeno con Hidrogeno en el centro de un hexagono

In [57]:
supercell_in = unit_cell.repeat((3,3,1)) # La función repeat((3, 3, 1)) repite la celda unitaria en las direcciones x e y para formar una supercelda de 3x3 hexágonos.
#view(supercell_in)

# Calcula el centro de un hexágono al tomar el promedio de las posiciones de dos átomos de carbono vecinos (índices 0 y 3).
# Luego, agrega un desplazamiento de 1.1 Å en la dirección z.

center_hex_position = (supercell_in[0].position + supercell_in[3].position) / 2 + np.array([0, 0, 1.1])

# Añadir los átomos de hidrógeno
# Usa la clase Atoms para definir un átomo de hidrógeno en la posición calculada y lo agrega a la supercelda.

supercell_in += Atoms("H", positions=[center_hex_position])


cell_params = [list(supercell_in.cell[0]),
				list(supercell_in.cell[1]),
				[0.0,0.0,15.0]]

supercell_in.cell[2][2] = 15.0

supercell_in.positions[:,2] += (cell_params[2][2]/2) - supercell_in.get_center_of_mass()[2]


view(supercell_in)

<Popen: returncode: None args: ['/home/hernan-barquero/anaconda3/bin/python'...>

In [58]:
# Genera un archivo de entrada para Quantum ESPRESSO con la configuración donde el hidrógeno está sobre el átomo de carbono.
write("graphene_h_in_c.in", supercell_in, format="espresso-in", pseudopotentials={
    "C": "C.pbe-n-kjpaw_psl.1.0.0.UPF",  # Especifica el pseudopotencial para carbono.
    "H": "H.pbe-kjpaw_psl.1.0.0.UPF"     # Especifica el pseudopotencial para hidrógeno.
}, input_data={
    "control": {
        "calculation": "relax",           # Indica que se hará una relajación de la estructura.
        "prefix": "graphene_h_over_c",    # Prefijo para los archivos de salida generados por Quantum ESPRESSO.
        "outdir": ".",             # Carpeta donde se guardarán los archivos temporales de Quantum ESPRESSO.
        "pseudo_dir": "/home/ebarquero/practicas/p2/pseudo/",          # Carpeta donde se encuentran los pseudopotenciales.
		"tstress": True,
		"tprnfor": True
    },
    "system": {
        "ecutwfc": 60,                    # Corte de energía para las funciones de onda (en Ry).
        "ecutrho": 600,                   # Corte de energía para la densidad de carga (en Ry).
        "occupations": "smearing",        # Usa un método de smearing para las ocupaciones electrónicas.
        "smearing": "mp",                 # Método de smearing: Methfessel-Paxton.
        "degauss": 0.02 ,                  # Ancho del smearing en Ry.
		"ibrav":0, 							# Define una celda generica
    },
    "electrons": {
        "conv_thr": 1e-6                  # Criterio de convergencia para las iteraciones electrónicas.
    }
}, kpts = (3,3,3),koffset = (0,0,0))